# 3장. 협업 필터링 추천 시스템

앞 장에서, 사용자 집단별 (성별, 직업별 등) 추천이 생각만큼 정확한 결과를 내지 못했다.

인구통계적 변수를 기준으로 나누지 않고 **비슷한 취향을 가진 사람들은 다른 아이템에 대해서도 비슷한 취향을 가지고 있을 것**이라고 가정한 추천 알고리즘

# 3.1 협업 필터링의 원리

- 기존적으로 취향이 비슷한 사람들의 집단이 존재한다고 가정

- 추천의 대상이 되는 어떤 한 사람이 있으면, 이 사람과 비슷한 사람들(neighbor)을 찾아내기만 하면, 이 사람들이 공통적으로 좋아하는 제품, 서비스를 추천대상인 사람에게 추천하면 된다.

<br>

[ 영화 추천의 예 ]

1. 우선 취향이 비슷한 사용자를 찾는다. 취향은 영화에 대한 평가로 나타날 것으로 각 사용자가 평가의 유사성을 계산한다.

2. 취향이 비슷한 사용자가 가장 좋게 평가한 영화를 찾는다. 아직 보지 않은 영화로 평점의 평균을 내어 평점평균이 높은 영화를 좋아할 것으로 예상한다.

# 3.2 유사도 지표

**CF는 사용자들의 평가를 기반으로 사용자 간 유사도를 구하는 것이 핵심이다.**

<br>
1. 상관계수

- 평가 자료가 연속값인 경우에 가장 이해하기 쉬운 유사도

- 두 사용자가 공통으로 평가한 아이템으로 판단하는 상관계수

- 최소 -1, 최대 1

- 이해하기 쉬운 유사도 측정치이기는 하지만 협업 필터링에서 하용하는 경우, 늘 좋은 결과를 가져오지는 못한다.

- 데이터가 이진값을 갖는다면 사용할 수 없다.

<br>
2. 코사인 유사도

- 평가 자료가 연속값인 경우에 널리 쓰이는 유사도

- 각 아이템을 하나의 차원으로 보고 사용자의 평가값을 좌표값으로 본다.

- 각 사용자의 평가값을 벡터로 해서 두 사용자 간의 벡터의 각도(코사인값)를 구할 수 있다.

- 두 사용자의 평가값이 유사할수록 각도가 작다(코사인값이 크다)는 것을 알 수 있다.

- 최소 -1, 최대 1

- 데이터가 이진값을 갖는다면 사용할 수 없다.

<br>
3. 타니모토 계수, 자카드 계수

- 데이터가 이진값을 갖는다면 쓰이는 유사도

- 최소 0, 최대 1

# 3.3 기본 CF 알고리즘

1) 모든 사용자 간의 평가의 유사도를 계산한다. 위에서 설명한 상관계수, 코사인 유사도 등을 사용할 수 있다.

2) 현재 추천 대상이 되는 사람과 다른 사용자의 유사도를 추출한다

3) 현재 사용자가 평가하지 않은 모든 아이템에 대해서 현재 사용자의 예상 평가값을 구한다. 예상 평가값은 다른 사용자의 해당 아이템에 대한 평가(평점)를 현재 사용자와 그 사용자와의 유사도를 가중해서 평균을 낸다.

4) 아이템 중에서 예상 평가값이 가장 높은 N개의 아이템을 추천한다.

In [4]:
import numpy as np
import pandas as pd

# 데이터 읽어 오기 
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('./Downloads/데이터/u.user', sep='|', names=u_cols, encoding='latin-1')
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown', 
          'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 
          'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
          'Thriller', 'War', 'Western']
movies = pd.read_csv('./Downloads/데이터/u.item', sep='|', names=i_cols, encoding='latin-1')
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('./Downloads/데이터/u.data', sep='\t', names=r_cols, encoding='latin-1')

# timestamp 제거 
ratings = ratings.drop('timestamp', axis=1)
# movie ID와 title 빼고 다른 데이터 제거
movies = movies[['movie_id', 'title']]

# train, test set 분리
from sklearn.model_selection import train_test_split
x = ratings.copy()
y = ratings['user_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)

# 정확도(RMSE)를 계산하는 함수
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

# 모델별 RMSE를 계산하는 함수 
def score(model):
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

# train 데이터로 Full matrix 구하기 
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')

In [12]:
# train set의 모든 가능한 사용자 pair의 Cosine similarities 계산
from sklearn.metrics.pairwise import cosine_similarity

matrix_dummy = rating_matrix.copy().fillna(0) # nan이 있으면 에러가 발생하므로 0으로 대체
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)

In [13]:
user_similarity

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.156764,0.037630,0.039673,0.318681,0.362615,0.338854,0.256949,0.085570,0.329857,...,0.310912,0.098990,0.182576,0.177688,0.173029,0.091343,0.278048,0.170281,0.158549,0.301717
2,0.156764,1.000000,0.070150,0.171366,0.059043,0.150307,0.068358,0.095238,0.099047,0.105557,...,0.100999,0.149052,0.265040,0.252594,0.160767,0.169169,0.123461,0.102931,0.154157,0.093445
3,0.037630,0.070150,1.000000,0.283313,0.029707,0.038773,0.064327,0.043453,0.025250,0.032668,...,0.034958,0.019882,0.088862,0.069124,0.160673,0.036387,0.088518,0.087930,0.134627,0.016242
4,0.039673,0.171366,0.283313,1.000000,0.013063,0.045687,0.054625,0.099073,0.077101,0.047882,...,0.024399,0.048569,0.110876,0.067543,0.076794,0.000000,0.158019,0.061372,0.198136,0.058192
5,0.318681,0.059043,0.029707,0.013063,1.000000,0.190078,0.206072,0.230621,0.038029,0.183692,...,0.292721,0.053652,0.051476,0.040256,0.091187,0.037551,0.213655,0.127706,0.116820,0.216765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.091343,0.169169,0.036387,0.000000,0.037551,0.091944,0.099939,0.098967,0.000000,0.052083,...,0.063980,0.303960,0.259678,0.221035,0.347246,1.000000,0.052340,0.118014,0.057917,0.116524
940,0.278048,0.123461,0.088518,0.158019,0.213655,0.271642,0.232203,0.205965,0.150726,0.216784,...,0.211196,0.121545,0.093604,0.046135,0.145703,0.052340,1.000000,0.121060,0.175002,0.166654
941,0.170281,0.102931,0.087930,0.061372,0.127706,0.128308,0.036578,0.085657,0.053602,0.102871,...,0.060961,0.187297,0.188644,0.123263,0.283625,0.118014,0.121060,1.000000,0.105851,0.096256


In [21]:
# 주어진 영화의 (movie_id) 가중평균 rating을 계산하는 함수, 
# 가중치는 주어진 사용자와 다른 사용자 간의 유사도(user_similarity)
def CF_simple(user_id, movie_id):
    if movie_id in rating_matrix:
        # 현재 사용자와 다른 사용자 간의 similarity 가져오기
        sim_scores = user_similarity[user_id].copy()
        # 현재 영화에 대한 모든 사용자의 rating값 가져오기
        movie_ratings = rating_matrix[movie_id].copy()
        # 현재 영화를 평가하지 않은 사용자의 index 가져오기
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index
        # 현재 영화를 평가하지 않은 사용자의 rating (null) 제거
        movie_ratings = movie_ratings.dropna()
        # 현재 영화를 평가하지 않은 사용자의 similarity값 제거
        sim_scores = sim_scores.drop(none_rating_idx)
        # 현재 영화를 평가한 모든 사용자의 가중평균값 구하기
        mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
    else:
        mean_rating = 3.0
    return mean_rating

# 정확도 계산
score(CF_simple)

1.0177461316868934

# 3.4 이웃을 고려한 CF

- 앞서 설명한 단순한 CF 알고리즘을 개선할 수 있는 한가지 방법은 **사용자 중에서 유사도가 높은 사용자를 선정해서 그 사람들의 평점만 가지고 예측하는 것이다.**

- 즉 이웃을 전체 사용자로 하는 대신에 유사도가 높은 사람만을 이웃으로 선정해서 이웃의 크기를 줄이는 것이다.

<br>

[ 이웃을 정하는 기준 ]

1. 이웃의 크기를 미리 정해놓고 추천 대상 사용자와 가장 유사한 K명을 선택하는 KNN 방법

2. 크기 대신 유사도의 기준(예를 들어 상관계수 0.8이상)을 정해 놓고 이 기준을 충족시키는 사용자를 이웃으로 정하는 Thresholding


- 일반적으로는 Thresholding 방법이 KNN보다 정확하기는 하지만 정해진 기준을 넘는 사용자가 없어서 추천을 하지 못하는 경우가 많기 때문에 KNN이 무난하게 많이 쓰인다.

In [22]:
# 모델별 RMSE를 계산하는 함수 
def score(model, neighbor_size=0):
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    y_pred = np.array([model(user, movie, neighbor_size) for (user, movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

In [23]:
##### (1) 

# Neighbor size를 정해서 예측치를 계산하는 함수 
def cf_knn(user_id, movie_id, neighbor_size=0):
    if movie_id in rating_matrix:
        # 현재 사용자와 다른 사용자 간의 similarity 가져오기
        sim_scores = user_similarity[user_id].copy()
        # 현재 영화에 대한 모든 사용자의 rating값 가져오기
        movie_ratings = rating_matrix[movie_id].copy()
        # 현재 영화를 평가하지 않은 사용자의 index 가져오기
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index
        # 현재 영화를 평가하지 않은 사용자의 rating (null) 제거
        movie_ratings = movie_ratings.drop(none_rating_idx)
        # 현재 영화를 평가하지 않은 사용자의 similarity값 제거
        sim_scores = sim_scores.drop(none_rating_idx)
##### (2) Neighbor size가 지정되지 않은 경우        
        if neighbor_size == 0:          
            # 현재 영화를 평가한 모든 사용자의 가중평균값 구하기
            mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
##### (3) Neighbor size가 지정된 경우
        else:                       
            # 해당 영화를 평가한 사용자가 최소 2명이 되는 경우에만 계산
            if len(sim_scores) > 1: 
                # 지정된 neighbor size 값과 해당 영화를 평가한 총사용자 수 중 작은 것으로 결정
                neighbor_size = min(neighbor_size, len(sim_scores))
                # array로 바꾸기 (argsort를 사용하기 위함)
                sim_scores = np.array(sim_scores)
                movie_ratings = np.array(movie_ratings)
                # 유사도를 순서대로 정렬
                user_idx = np.argsort(sim_scores)
                # 유사도를 neighbor size만큼 받기
                sim_scores = sim_scores[user_idx][-neighbor_size:]
                # 영화 rating을 neighbor size만큼 받기
                movie_ratings = movie_ratings[user_idx][-neighbor_size:]
                # 최종 예측값 계산 
                mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
            else:
                mean_rating = 3.0
    else:
        mean_rating = 3.0
    return mean_rating

# 정확도 계산
score(cf_knn, neighbor_size=30)


##### (4) 주어진 사용자에 대해 추천을 받기 
# 전체 데이터로 full matrix와 cosine similarity 구하기 
rating_matrix = ratings.pivot_table(values='rating', index='user_id', columns='movie_id')
from sklearn.metrics.pairwise import cosine_similarity
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)

def recom_movie(user_id, n_items, neighbor_size=30):
    # 현 사용자가 평가한 영화 가져오기
    user_movie = rating_matrix.loc[user_id].copy()
    for movie in rating_matrix:
        # 현 사용자가 이미 평가한 영화는 제외 (평점을 0으로)        
        if pd.notnull(user_movie.loc[movie]):
            user_movie.loc[movie] = 0
        # 현 사용자가 평가하지 않은 영화의 예상 평점 계산
        else:
            user_movie.loc[movie] = cf_knn(user_id, movie, neighbor_size)
    # 영화를 예상 평점에 따라 정렬해서 제목을 뽑아서 돌려 줌
    movie_sort = user_movie.sort_values(ascending=False)[:n_items]
    recom_movies = movies.loc[movie_sort.index]
    recommendations = recom_movies['title']
    return recommendations

recom_movie(user_id=2, n_items=5, neighbor_size=30)


##### (5) 최적의 neighbor size 구하기

# train set으로 full matrix와 cosine similarity 구하기 
rating_matrix = x_train.pivot_table(values='rating', index='user_id', columns='movie_id')
from sklearn.metrics.pairwise import cosine_similarity
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)
for neighbor_size in [10, 20, 30, 40, 50, 60]:
    print("Neighbor size = %d : RMSE = %.4f" % (neighbor_size, score(cf_knn, neighbor_size)))


Neighbor size = 10 : RMSE = 1.0276
Neighbor size = 20 : RMSE = 1.0117
Neighbor size = 30 : RMSE = 1.0086
Neighbor size = 40 : RMSE = 1.0085
Neighbor size = 50 : RMSE = 1.0090
Neighbor size = 60 : RMSE = 1.0097
